In [29]:
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn
import tqdm
import matplotlib 

In [5]:
num_stacks = 6
d_model = 512

In [7]:
class Encoder(nn.Module):
  def __init__(self):
    super().__init__()
    # Sub Layer 1 -> multi-head self-attention

    # Residual connection 

    # Layer Normalization

    # Sub Layer 2 -> fully connected feed-forward network

    # Residual connection 

    # Layer Normalization

In [8]:
class Decoder(nn.Module):
  def __init__(self):
    super().__init__()

    # Sub Layer 1 -> masked multi-head attention over the output of the encoder stack
    # masked to ensure only looking at previous known outputs

    # Residual connection 

    # Layer Normalization

    # Sub Layer 2 -> multi-head self-attention

    # Residual connection 

    # Layer Normalization

    # Sub Layer 3 -> fully connected feed-forward network

    # Residual connection 

    # Layer Normalization

### Attention

def attention(query, key, val, model_dim) -> output

- Query, keys, values, and output are all vectors.

Output
- weighted sum of the values
- weight assened to each value = compatibility_fn(query, corresponding_key)



In [11]:
h = 8
def multi_head_attention():
  pass

In [13]:
class TransformerBlock(nn.Module):
  def __init__(self, N=6, h=8, d_model=512, d_ff=2048, p_drop=0.1, eps_ls=0.1):
    """
    N = number of layers
    h = number of parallel attention layers (heads)
    d_k = dimension of embeddings
    d_ff = dimension of feed forwards
    p_drop = percentage of dropout
    eps_ls = label smoothing value (improves accuracy but hurts perplexity)
    """
    super().__init__()
    self.num_heads = h
    self.d_model = d_model
    self.d_k = d_model // self.num_heads
    
    self.dropout_num = p_drop
    self.ff_1 = nn.Linear(d_model, d_ff)
    self.ff_2 = nn.Linear(d_model, d_ff)

    

    self.dropout = nn.Dropout(p_drop)
    self.out = nn.Linear(d_model, d_model)

  def attention(self, embed_dim):

    # x = q @ v
    # x = scale
    # x = optional mask
    # x = softmax(x)
    # return x @ v
    # each q,k,v is a linear neural network
    # <q,k> tells us how much we want to attend to certain features
    # v tells us what to attend to 
    # like database in which we use a query that helps us find a key which has a corresponding value
    # We just want to learn a way to make sure our queries grab the appropriate key.

    self.query = nn.Linear(self.d_model, self.d_model)
    self.key = nn.Linear(self.d_model, self.d_model)
    self.value = nn.Linear(self.d_model, self.d_model)

    inner = query @ key.T
    inner /= np.sqrt(self.dim_size) # large d_k tends to very small gradients, this helps counteract that
    weights = torch.nn.Softmax(inner) # turns into a probability distribution
                                      # If our attention score is 0 then we don’t pay attention to those parts
    attention = torch.dot(weights,value)

    return attention

  def forward(self, x):
    x = x + self.attn(x).dropout(0.1)
    x = x.layernorm().linear(*self.ln1)
    x = x + self.act(x.linear(*self.ff1)).linear(*self.ff2).dropout(0.1)
    x = x.layernorm().linear(*self.ln2)

    return x

In [ ]:
class Attention(nn.Module):
   def __init__(self, heads, d_model, dropout = 0.1):
        super().__init__()

        # x = q @ v
        # x = scale
        # x = optional mask
        # x = softmax(x)
        # return x @ v
        # each q,k,v is a linear neural network
        # <q,k> tells us how much we want to attend to certain features
        # v tells us what to attend to 
        # like database in which we use a query that helps us find a key which has a corresponding value
        # We just want to learn a way to make sure our queries grab the appropriate key.

        self.query = nn.Linear(self.d_model, self.d_model)
        self.key = nn.Linear(self.d_model, self.d_model)
        self.value = nn.Linear(self.d_model, self.d_model)

        inner = query @ key.T
        inner /= np.sqrt(self.dim_size) # large d_k tends to very small gradients, this helps counteract that
        weights = torch.nn.Softmax(inner) # turns into a probability distribution
                                          # If our attention score is 0 then we don’t pay attention to those parts
        attention = torch.dot(weights,value)

        return attention

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff=2048, dropout = 0.1):
        super().__init__() 
        # We set d_ff as a default to 2048
        self.linear_1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model)
    def forward(self, x):
        x = self.dropout(F.relu(self.linear_1(x)))
        x = self.linear_2(x)
        return x

In [30]:
class Embedder(nn.Module):
    def __init__(self, vocab_size, d_model):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, d_model)
    def forward(self, x):
        return self.embed(x)

In [14]:
class Transformer(nn.Module):
  def __init__(self, num_layers, embed_dim, num_heads, ff_dim):
    super().__init()
    self.core = []

    for i in range(num_layers):
      self.core.append(TransformerBlock(embed_dim, num_heads, ff_dim))
      
    self.final = nn.Linear(embed_dim, ff_dim)
